In [ ]:
import os
if str.lower(dbutils.widgets.getAll().get("install_ml_libs", "False")) == "true":
  os.system("pip install -qqqq lightgbm databricks-feature-store bayesian-optimization mlflow hyperopt shap databricks-automl-runtime")
  dbutils.library.restartPython()

In [ ]:
import json
import os

def _get_config():

  # Config should be stored at <solution_accelerator_root>/notebooks/demo_setup/config.json
  # We are expecting _init.ipynb to be run from somewhere within the notebooks folder
  # Databricks will set the current working directory to currently running notebook
  # This will traverse up the folder structure until it finds the notebooks folder
  notebook_root = os.getcwd()
  while not notebook_root.endswith('/notebooks'):
      parent = os.path.dirname(notebook_root)
      if parent == notebook_root:
          break
      notebook_root = parent
  notebook_root
  json_config_path = os.path.join(notebook_root, "demo_setup/config.json")

  # The DABS setup job should inject the config as job params, and set the load_config_from_file param to False
  # If this check is absent, we'll assume the user is running the notebook interactively and load the config from the file
  config = dbutils.widgets.getAll()
  if str.lower(config.get("load_config_from_file", "True")) == "true":
    try:
      with open(json_config_path, "r") as f:
        config = json.load(f)
    except:
      # If we can't load the config from the file, assume that the setup process wasn't run
      print(f"❌ Could not load config from {json_config_path}")
      print(f"Make sure that you have:")
      print(f"   1. Deployed this Solution Accelerator using Databricks Asset Bundles from the UI")
      print(f"   2. Run the deploy-iops-demo-workflow job, which will create the config.json file")
      print(f"If you are deploying the bundle from the CLI, you will need to manually configure the config.json file")
      print(f"Expected paramters are:")
      print(f"   1. catalog_name")
      print(f"   2. schema_name")
      print(f"   3. volume_name")
      raise Exception(
        "Could not load config. Please ensure you have run the setup job (deploy-iops-demo-workflow) created by Databricks Asset Bundles")
  else:
    with open(json_config_path, "w") as f:
      json.dump(config, f)

  return config

config = _get_config()

catalog_name = config['catalog_name']
schema_name = config['schema_name']
volume_name = config['volume_name']

print(f"✅ Variables configured:")
print(f"   📁 Catalog: {catalog_name}")
print(f"   📂 Schema: {schema_name}")
print(f"   💾 Volume: {volume_name}")

In [ ]:
### Helper functions for the demo ###

In [ ]:
dbx_host_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()

In [ ]:
def drop_fs_table(table_name):
  from databricks.feature_store import FeatureStoreClient
  fs = FeatureStoreClient()
  try:
    fs.drop_table(table_name)  
  except Exception as e:
    print(f"Can't drop the fs table, probably doesn't exist? {e}")
  try:
    spark.sql(f"DROP TABLE IF EXISTS `{table_name}`")
  except Exception as e:
    print(f"Can't drop the delta table, probably doesn't exist? {e}")

In [ ]:

def display_link(url, text=None, description=None):
    displayHTML(f"""
            <b style='font-size:24px;'>{description if description else ""}</b><a href='{url}' target='_blank' style='font-size:24px;'>{text if text else url}</a>
    """)